In [1]:
#------------------------------------------------------------------------------+
#   Selected Topics in Computational Intelligence I (COMP8038041)              #
#   Artificial Bee Colony (ABC) Algorithm with Python                          #
#   Kelas : LTY1                                                               #
#   Dosen : Amalia Zahra, S.Kom.,Ph.D.                                         #
#------------------------------------------------------------------------------+

In [2]:
import numpy as np
import random as rand
import time

In [3]:
class ArtificialBeeColony:

    def __init__(self, D, Lb, Ub, n, generation, ans, func):
        self.D = D
        self.Lb = Lb
        self.Ub = Ub
        self.n = n
        self.generation = generation
        self.ans = ans
        self.func = func
        self.bestFunc = 0
        self.bestFoodSourceArray = np.zeros(D)
        self.foodSourceArray = np.ones((n,D))
        self.tmpFoodSourceArray = np.ones(D)
        self.funcArray = np.ones(n)
        self.fitnessArray = np.ones(n)
        self.trialArray = np.zeros(n)
        self.pArray = np.zeros(n)
        
    def fitness(self,X):
        if(X >= 0):
            return 1/(1 + X)
        else:
            return 1 + abs(X)

    def generateNew(self,X,Xp):
        Xnew = X + rand.uniform(-1,1)*(X - Xp)
        if(Xnew < self.Lb): return self.Lb
        elif(Xnew > self.Ub): return self.Ub
        else: return Xnew

    def updateSolution(self,index):
        randVariableToChange = rand.randint(0,self.D-1)
        randPartner = index
        while(randPartner == index):
            randPartner = rand.randint(0,self.n-1)

        for j in range(self.D):
            self.tmpFoodSourceArray[j] = self.foodSourceArray[index][j]
        self.tmpFoodSourceArray[randVariableToChange] = self.generateNew(self.foodSourceArray[index][randVariableToChange], self.foodSourceArray[randPartner][randVariableToChange])
        
        if(self.ans == 0): 
            oriVal = self.funcArray[index]
            newVal = self.func(self.tmpFoodSourceArray,self.D)
        elif(self.ans == 1):
            oriVal = self.fitnessArray[index]
            newVal = self.fitness(self.func(self.tmpFoodSourceArray,self.D))

        if(newVal < oriVal):
            self.foodSourceArray[index][randVariableToChange] = self.tmpFoodSourceArray[randVariableToChange]
            self.funcArray[index] = self.func(self.tmpFoodSourceArray,self.D)
            self.fitnessArray[index] = self.fitness(self.func(self.tmpFoodSourceArray,self.D))
            self.trialArray[index] = 0
        else:
            self.trialArray[index] = self.trialArray[index] + 1
            
    def printLocalBestSolution_MAX(self):
        localBest = int(np.where(self.funcArray == self.funcArray.max())[0])
        print("Local Best Food Source:", self.foodSourceArray[localBest])
        print("local Best F(x) =",self.funcArray.max())
        if(self.funcArray.max() > self.bestFunc):
            for i in range(self.D):
                self.bestFoodSourceArray[i] = self.foodSourceArray[localBest][i]
            self.bestFunc = self.funcArray.max()
            
    def printLocalBestSolution_MIN(self):
        localBest = int(np.where(self.funcArray == self.funcArray.min())[0])
        print("Local Best Food Source:", self.foodSourceArray[localBest])
        print("local Best F(x) =",self.funcArray.min())
        if(self.funcArray.min() < self.bestFunc):
            for i in range(self.D):
                self.bestFoodSourceArray[i] = self.foodSourceArray[localBest][i]
            self.bestFunc = self.funcArray.min()
        
    def printCurrentSolution(self):
        print("==================================")
        print("foodSourceArray\n", self.foodSourceArray)
        print("funcArray\n", self.funcArray)
        print("fitnessArray\n", self.fitnessArray)
        print("trialArray\n", self.trialArray)
        print("==================================")
    
    def init(self):
        if(self.ans == 0): self.bestFunc = float("inf")
        elif(self.ans == 1): self.bestFunc = -float("inf")
            
        for i in range(self.n):
            for j in range(self.D):
                self.foodSourceArray[i][j] = rand.uniform(self.Lb, self.Ub)
            self.funcArray[i] = self.func(self.foodSourceArray[i,:],self.D)
            self.fitnessArray[i] = self.fitness(self.funcArray[i])
            
    def doRun(self):
        start = time.time()
        self.init()
            
        for gen in range(self.generation):
            print("Generation:", gen+1)

            #Employed Bee Phase
            for i in range(self.n):
                self.updateSolution(i)

            #Onlooker Bee Phase
            for i in range(self.n):
                self.pArray[i] = self.fitnessArray[i]/self.fitnessArray.sum()
            for i in range(self.n):
                if(rand.random() < self.pArray[i]):
                    self.updateSolution(i)
            
            if(self.ans == 0): self.printLocalBestSolution_MIN()
            elif(self.ans == 1): self.printLocalBestSolution_MAX()
            
            #Scout Bee Phase
            limit = 1
            for i in range(self.n):
                if(self.trialArray[i] > limit):
                    for j in range(self.D):
                        self.foodSourceArray[i][j] = rand.uniform(self.Lb, self.Ub)
                    self.funcArray[i] = self.func(self.foodSourceArray[i,:],self.D)
                    self.fitnessArray[i] = self.fitness(self.funcArray[i])
                    self.trialArray[i] = 0
        end = time.time()
        print("============================================")
        print("Time Consume：%f " % (end - start))
        print("Best Food Source:", self.bestFoodSourceArray)
        print("Best F(x) =", self.bestFunc)

In [6]:
def test(X,D):
    x1 = X[0]
    x2 = X[1]
    return x1**2 - x1*x2 + x2**2 + 2*x1 + 4*x2 + 3

In [7]:
# ArtificialBeeColony(D, Lb, Ub, n, generation, ans[min=0/max=1], func)
abc = ArtificialBeeColony(2, -5, 5, 10, 20, 1, test)
abc.doRun()

Generation: 1
Local Best Food Source: [ 5.         -4.85659226]
local Best F(x) = 66.4430806189561
Generation: 2
Local Best Food Source: [ 5. -5.]
local Best F(x) = 68.0
Generation: 3
Local Best Food Source: [ 5. -5.]
local Best F(x) = 68.0
Generation: 4
Local Best Food Source: [ 5. -5.]
local Best F(x) = 68.0
Generation: 5
Local Best Food Source: [5.         4.11251444]
local Best F(x) = 50.80026058720156
Generation: 6
Local Best Food Source: [5.         4.11251444]
local Best F(x) = 50.80026058720156
Generation: 7
Local Best Food Source: [5.         4.33616899]
local Best F(x) = 52.46619249111179
Generation: 8
Local Best Food Source: [-1.95462475  4.78716316]
local Best F(x) = 54.3339997742307
Generation: 9
Local Best Food Source: [5.         4.79260514]
local Best F(x) = 56.17645885743361
Generation: 10
Local Best Food Source: [ 5.         -4.71442303]
local Best F(x) = 64.94020757486228
Generation: 11
Local Best Food Source: [-3.27952859  5.        ]
local Best F(x) = 68.5938935194

In [8]:
def RastriginFunc(X, D):
    funsum = 0
    for i in range(D):
        x = X[i]
        funsum += x**2-10*np.cos(2*np.pi*x)
    funsum += 10*D
    return funsum

In [9]:
# ArtificialBeeColony(D, Lb, Ub, n, generation, ans[min=0/max=1], func)
abcRast = ArtificialBeeColony(2, -5, 5, 5, 100, 1, RastriginFunc)
abcRast.doRun()

Generation: 1
Local Best Food Source: [-4.55880806  3.35025935]
local Best F(x) = 67.22307015964866
Generation: 2
Local Best Food Source: [-4.55880806  3.35025935]
local Best F(x) = 67.22307015964866
Generation: 3
Local Best Food Source: [4.49544369 2.48301018]
local Best F(x) = 66.31333205454764
Generation: 4
Local Best Food Source: [4.49544369 2.48301018]
local Best F(x) = 66.31333205454764
Generation: 5
Local Best Food Source: [-4.7036526   4.70550052]
local Best F(x) = 69.89688870592794
Generation: 6
Local Best Food Source: [-4.7036526   4.70550052]
local Best F(x) = 69.89688870592794
Generation: 7
Local Best Food Source: [3.40856075 4.44811041]
local Best F(x) = 69.27165940521637
Generation: 8
Local Best Food Source: [3.40856075 4.48625033]
local Best F(x) = 70.1019135120761
Generation: 9
Local Best Food Source: [3.40856075 4.48625033]
local Best F(x) = 70.1019135120761
Generation: 10
Local Best Food Source: [-2.40268194  4.6440662 ]
local Best F(x) = 61.70362144793511
Generation:

In [10]:
def StyblinskiTangFunc(X, D):
    funsum = 0
    for i in range(D):
        x = X[i]
        funsum += (x**4) - 16*(x**2) + 5*x
    funsum *= 0.5
    return funsum

In [11]:
# ArtificialBeeColony(D, Lb, Ub, n, generation, ans[min=0/max=1], func)
abcTang = ArtificialBeeColony(2, -5, 5, 5, 100, 0, StyblinskiTangFunc)
abcTang.doRun()

Generation: 1
Local Best Food Source: [ 2.80377389 -3.42963676]
local Best F(x) = -58.477159125338275
Generation: 2
Local Best Food Source: [ 2.80377389 -3.42963676]
local Best F(x) = -58.477159125338275
Generation: 3
Local Best Food Source: [-2.7318183   1.58661003]
local Best F(x) = -51.688918014709486
Generation: 4
Local Best Food Source: [-2.18468077 -2.10786447]
local Best F(x) = -63.1982551267855
Generation: 5
Local Best Food Source: [-2.73233368 -2.10786447]
local Best F(x) = -69.63193387446182
Generation: 6
Local Best Food Source: [-2.73233368 -2.10786447]
local Best F(x) = -69.63193387446182
Generation: 7
Local Best Food Source: [-2.76341665 -2.10786447]
local Best F(x) = -69.7863312287806
Generation: 8
Local Best Food Source: [-2.76341665 -2.10786447]
local Best F(x) = -69.7863312287806
Generation: 9
Local Best Food Source: [-2.76341665 -2.51615076]
local Best F(x) = -75.7400860291389
Generation: 10
Local Best Food Source: [-2.76341665 -2.51615076]
local Best F(x) = -75.74008